## Difference Project

Some useful utility function.

In [1]:
function SoftThreshold(z, lambda)
    abs(z) < lambda ? 0. : z > 0 ? z - lambda : z + lambda
end

@show SoftThreshold(3, 1)
@show SoftThreshold(-3, 1)
@show SoftThreshold(-0.5, 1);

SoftThreshold(3,1) => 2
SoftThreshold(-3,1) => -2
SoftThreshold(-0.5,1) => 0.0


Generate data and compute precision matrices

In [2]:
p = 30
Sigmax = eye(p,p)
Sigmay = eye(p,p)
Sigmay[1,2] = 0.5
Sigmay[2,1] = 0.5

sqmy = sqrtm(Sigmay)

n = 1000
X = randn(n,p)
Y = randn(n,p) * sqmy;

hSx = cov(X, corrected=false)
hSy = cov(Y, corrected=false);

Solve the problem using Convex and Mosek for comparison purposes

In [3]:
using Convex
using Mosek

solver = MosekSolver(LOG=1)
set_default_solver(solver);

In [23]:
lambda = 0.2

Delta = Variable(p,p);
prob = minimize(quad_form(vec(Delta), kron(hSy,hSx)) / 2 - trace((hSy-hSx)*Delta) +  lambda * norm(vec(Delta), 1))
prob.constraints += [Delta == Delta']
@time solve!(prob)

sol = Delta.value

Computer
  Platform               : Linux/64-X86    
  Cores                  : 4               

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 3606            
  Cones                  : 2               
  Scalar variables       : 2707            
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Conic interior-point optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Total number of eliminations : 901
Eliminator terminated.
Eliminator started.
Total number of eliminations : 901
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.00            
Eliminator - elim's                 : 901             
Lin. dep.  - tries                  : 1                 time    

30x30 Array{Float64,2}:
 -0.00088797   0.257462    3.59289e-7  …   2.20578e-7   3.34436e-7
  0.257462    -4.12929e-7  1.98721e-7      2.81931e-7   4.50355e-7
  3.59393e-7   1.98733e-7  2.37406e-7      3.28745e-7   1.68113e-7
  3.96692e-7   2.85966e-7  3.69161e-7      3.70942e-7   2.74786e-7
  2.53321e-7   8.97013e-8  4.58873e-7      2.34008e-7   3.02452e-7
  1.45265e-7   1.89976e-7  2.44157e-7  …   3.42738e-7   3.72059e-7
 -1.99219e-7   6.96321e-9  2.20553e-7      2.39401e-7   1.23979e-7
 -1.37885e-7   2.27545e-7  3.14888e-7      1.13687e-7   2.30467e-7
  6.56971e-7   3.45959e-7  2.71429e-7     -1.72423e-7   2.93402e-7
 -1.20126e-7   1.71192e-7  3.16529e-7      1.46532e-7   1.83718e-7
  3.87697e-7   2.7373e-7   5.36332e-7  …   3.08515e-7   2.35487e-7
  1.87029e-7   1.69441e-7  5.45685e-7      1.46594e-7   2.81818e-7
  5.28771e-7   3.72469e-7  2.65387e-7      2.15448e-7   3.33824e-8
  ⋮                                    ⋱                          
  3.40484e-7   8.1304e-7   2.78149e-7 

Shooting algorithm for the objective

In [24]:
function DiffCovShooting(hSx, hSy, lambda, Ups, options)    
    # process options    
    maxIter = 2000
    optTol = 1e-5
    
    #
    p = size(hSx, 1)
        
    Delta = zeros(p, p)
    A = zeros(p, p)
    
    iter = 1;
    while iter < maxIter    
        if mod(iter, 10) == 0
            @show "outer $iter"
        end
    
        fDone = true
        for a=1:p
            for b=a:p
                if a==b
                    # diagonal elements
                    x0 = hSx[a,a]*hSy[a,a]/2
                    x1 = A[a,a] - hSy[a,a] + hSx[a,a]
                    tmp = SoftThreshold(-x1/x0/2 + Delta[a,b], lambda*Ups[a,b] / 2 / x0)
                else
                    # off-diagonal elements
                    x0 = (hSx[a,a]*hSy[b,b] + hSx[b,b]*hSy[a,a])/2 + hSx[a,b]*hSy[a,b]
                    x1 = A[a,b] + A[b,a] - 2*(hSy[a,b] - hSx[a,b])
                    tmp = SoftThreshold(-x1/x0/2 + Delta[a,b], lambda*Ups[a,b] / x0)
                end
                
                h = tmp - Delta[a,b]
                Delta[a,b] = tmp
                Delta[b,a] = tmp
                if abs(h) > optTol
                    fDone = false
                end
                for j=1:p
                    for k=1:p
                        if a == b
                            A[j,k] = A[j,k] + h * hSx[j,a]*hSy[a,k]
                        else                            
                            A[j,k] = A[j,k] + h * (hSx[j,a]*hSy[b,k] + hSx[j,b]*hSy[a,k])
                        end
                    end
                end
            end
        end
        
        iter = iter + 1;
        if fDone
            break
        end
    end
    Delta
end

DiffCovShooting (generic function with 1 method)

In [25]:
@time solShoot = DiffCovShooting(hSx, hSy, lambda, ones(p,p), 1)

elapsed time: 0.043884328 seconds (1318760 bytes allocated)


30x30 Array{Float64,2}:
 -0.000128943  0.257209  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.257209     0.0       0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0          0.0       0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0          0.0       0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0          0.0       0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0          0.0       0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0          0.0       0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0          0.0       0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0          0.0       0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0          0.0       0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0          0.0       0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0          0.0       0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0          0.0       0.0  0.0  0.0     0.0  0.0 

In [26]:
@show dot(vec(sol), kron(hSy,hSx)*vec(sol)) / 2 - trace((hSy-hSx)*sol) +  lambda * norm(vec(sol), 1)
@show dot(vec(solShoot), kron(hSy,hSx)*vec(solShoot)) / 2 - trace((hSy-hSx)*solShoot) +  lambda * norm(vec(solShoot), 1);

(dot(vec(sol),kron(hSy,hSx) * vec(sol)) / 2 - trace((hSy - hSx) * sol)) + lambda * norm(vec(sol),1) => -0.06669806710727805
(dot(vec(solShoot),kron(hSy,hSx) * vec(solShoot)) / 2 - trace((hSy - hSx) * solShoot)) + lambda * norm(vec(solShoot),1) => -0.06674410092749669
